## LangChain tools
Tools w LangChain to zewnętrzne funkcje lub usługi (np. wyszukiwarka, kalkulator, API), które agent może wywoływać podczas rozwiązywania zadań, aby rozszerzyć swoje możliwości poza samo generowanie tekstu.

### Instalacja bibliotek

In [1]:
!pip install -q langchain langchain-openai python-dotenv

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Model gotowy.")


Model gotowy.


### Proste narzędzie (kalkulator)

In [3]:
from langchain.tools import tool

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Mnoży dwie liczby całkowite."""
    return x * y

print(multiply_numbers.invoke({"x": 6, "y": 7}))


42


### Użycie narzędzia z LLM

In [7]:
from langchain.agents import initialize_agent, AgentType

tools = [multiply_numbers]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Model sam zdecyduje, że musi użyć multiply_numbers
agent.run("Ile wynosi dwukrotność liczby ludzi na Ziemi?")




> Entering new AgentExecutor chain...
Thought: Najpierw muszę ustalić aktualną liczbę ludzi na Ziemi. Szacuje się, że liczba ta wynosi około 8 miliardów. Następnie obliczę dwukrotność tej liczby.

Action:
```
{
  "action": "multiply_numbers",
  "action_input": {
    "x": 8,
    "y": 2
  }
}
```

Observation: 16
Thought:Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów."
}
```


> Finished chain.


'Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów.'

### Kolejne narzędzie (prosta baza wiedzy)

In [5]:
@tool
def company_info(name: str) -> str:
    """Zwraca podstawowe informacje o firmie na podstawie nazwy."""
    companies = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT.",
        "LangChain": "Biblioteka ułatwiająca budowanie aplikacji z modelami językowymi.",
        "Tesla": "Producent samochodów elektrycznych i baterii."
    }
    return companies.get(name, "Nie znam tej firmy.")

tools = [company_info]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Powiedz mi, czym zajmuje się firma Tesla.")




> Entering new AgentExecutor chain...
Muszę uzyskać podstawowe informacje o firmie Tesla, aby dowiedzieć się, czym się zajmuje.  
Action: company_info  
Action Input: "Tesla"  
Observation: Producent samochodów elektrycznych i baterii.
Thought:I now know the final answer  
Final Answer: Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.

> Finished chain.


'Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.'